In [1]:
!pip install -qU langchain-openai langchain-community langgraph langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install -q sentence-transformers

In [7]:
!pip install langchain_huggingface

In [10]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 88.3 MB/s eta 0:00:00


In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langgraph.graph import StateGraph, END
from langchain.tools import tool


os.environ["GROQ_API_KEY"] = "enter your api key"

# Dictionary
knowledge_base = {
    "pricing_and_features": {
        "basic_plan": {
            "price": "$29/month",
            "videos": "10 videos/month",
            "resolution": "720p resolution"
        },
        "pro_plan": {
            "price": "$79/month",
            "videos": "Unlimited videos",
            "resolution": "4K resolution",
            "extra": "AI captions"
        }
    },
    "company_policies": {
        "refunds": "No refunds after 7 days",
        "support": "24/7 support available only on Pro plan"
    }
}

# text chunks for RAG
kb_chunks = [
    f"Basic Plan: {knowledge_base['pricing_and_features']['basic_plan']['price']}, "
    f"{knowledge_base['pricing_and_features']['basic_plan']['videos']}, "
    f"{knowledge_base['pricing_and_features']['basic_plan']['resolution']}",

    f"Pro Plan: {knowledge_base['pricing_and_features']['pro_plan']['price']}, "
    f"{knowledge_base['pricing_and_features']['pro_plan']['videos']}, "
    f"{knowledge_base['pricing_and_features']['pro_plan']['resolution']}, "
    f"{knowledge_base['pricing_and_features']['pro_plan']['extra']}",

    f"Company Policies: Refunds - {knowledge_base['company_policies']['refunds']}. "
    f"Support - {knowledge_base['company_policies']['support']}."
]

# RAG Setup
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

vectorstore = FAISS.from_texts(kb_chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=512
)

# chains
intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intent classifier. Reply **ONLY** with one of these exact labels:\n"
               "casual_greeting\nproduct_pricing_inquiry\nhigh_intent_lead\n"
               "No explanation, no extra text."),
    MessagesPlaceholder("messages"),
])

intent_chain = intent_prompt | llm | StrOutputParser()

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using **only** the following context. Be concise and friendly:\n{context}"),
    MessagesPlaceholder("messages"),
])

rag_chain = (
    RunnableParallel(
        context=RunnableLambda(lambda state: state["messages"][-1].content) | retriever,
        messages=RunnableLambda(lambda state: state["messages"]), # Corrected: Pass only the list of messages
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)


@tool
def mock_lead_capture(name: str, email: str, platform: str):
    """Call this function ONLY after collecting name, email and platform."""
    print(f"\n>>> LEAD CAPTURED SUCCESSFULLY: {name} | {email} | {platform}")
    return "Lead captured successfully!"

tools = [mock_lead_capture]


class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], "add_messages"]
    intent: str
    lead_details: dict
    lead_collected: bool

def classify_intent(state: AgentState) -> dict:
    last_msg = state["messages"][-1]
    raw = intent_chain.invoke({"messages": [last_msg]})
    intent = raw.strip().lower()
    valid = {"casual_greeting", "product_pricing_inquiry", "high_intent_lead"}
    return {"intent": intent if intent in valid else "product_pricing_inquiry"}

def update_lead_details(state: AgentState) -> dict:
    if state["intent"] != "high_intent_lead":
        return {}

    details = state.get("lead_details", {})
    last_input = state["messages"][-1].content.strip()

    if "name" not in details:
        details["name"] = last_input
    elif "email" not in details:
        details["email"] = last_input
    elif "platform" not in details:
        details["platform"] = last_input

    return {"lead_details": details}

def handle_greeting(state: AgentState) -> dict:
    resp = llm.invoke([
        SystemMessage("Reply casually and warmly to a greeting."),
        *state["messages"]
    ]).content
    return {"messages": state["messages"] + [AIMessage(content=resp)]}

def handle_inquiry(state: AgentState) -> dict:
    resp = rag_chain.invoke(state)
    return {"messages": state["messages"] + [AIMessage(content=resp)]}

def handle_high_intent(state: AgentState) -> dict:
    details = state.get("lead_details", {})
    history = state["messages"]

    if state.get("lead_collected", False):
        return {"messages": history + [AIMessage(content="Thank you! Your information has been received. ")]}

    if "name" not in details:
        return {"messages": history + [AIMessage(content="Great! Could you please tell me your full name?")]}

    if "email" not in details:
        return {"messages": history + [AIMessage(content="Thanks! What's your email address?")]}

    if "platform" not in details:
        return {"messages": history + [AIMessage(content="Perfect! Which platform do you create content on? (e.g. YouTube, Instagram, TikTok...)?")]}


    result = mock_lead_capture.invoke(details)
    return {
        "lead_collected": True,
        "messages": history +
            [
            AIMessage(content="Thank you! We've successfully captured your details. Our team will reach out soon! 🚀"),
            ToolMessage(content=result, tool_call_id="lead_capture")
        ]
    }


def router(state: AgentState):
    intent = state["intent"]
    mapping = {
        "casual_greeting": "greeting",
        "product_pricing_inquiry": "inquiry",
        "high_intent_lead": "high_intent"
    }
    return mapping.get(intent, END)


workflow = StateGraph(AgentState)

workflow.add_node("classify", classify_intent)
workflow.add_node("update_details", update_lead_details)
workflow.add_node("greeting", handle_greeting)
workflow.add_node("inquiry", handle_inquiry)
workflow.add_node("high_intent", handle_high_intent)

workflow.set_entry_point("classify")
workflow.add_edge("classify", "update_details")
workflow.add_conditional_edges("update_details", router, {
    "greeting": "greeting",
    "inquiry": "inquiry",
    "high_intent": "high_intent"
})

workflow.add_edge("greeting", END)
workflow.add_edge("inquiry", END)
workflow.add_edge("high_intent", END)

app = workflow.compile()

def run_chat():
    state = {
        "messages": [],
        "lead_details": {},
        "lead_collected": False
    }

    print("AutoStream AI Agent is ready!  (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye! 👋")
            break

        state["messages"].append(HumanMessage(content=user_input))

        result = app.invoke(state)
        state.update(result)


        last = result["messages"][-1]
        if isinstance(last, (AIMessage, ToolMessage)):
            print("Agent:", last.content)

if __name__ == "__main__":
    run_chat()

AutoStream AI Agent is ready!  (type 'exit' to quit)

You: what are your pricing models?
Agent: We have two plans: 
1. Basic Plan: $29/month (10 videos, 720p resolution)
2. Pro Plan: $79/month (Unlimited videos, 4K resolution, AI captions)
You: i want to try the pro plan
Agent: Thanks! What's your email address?
You: anubhav10b@gmail.com
Agent: You've chosen the Pro Plan: $79/month (Unlimited videos, 4K resolution, AI captions). I'll assume you're all set. If you need help, feel free to ask.
You: no
Agent: I'll send a confirmation email to anubhav10b@gmail.com with a link to complete the payment and get started with the Pro Plan. Sound good?
You: yes
Agent: I've sent the email. Just click on the link, fill out the payment details, and you'll be all set to start using the Pro Plan. If you have any questions or need help, don't hesitate to reach out. Have a great day!
You: thanks
Agent: You're welcome. Enjoy the Pro Plan and happy creating. If you need anything, I'm here to help. Have a 